In [ ]:
import sys
sys.path.insert(0,'..')
sys.path.insert(0,'/home/ikea/miniconda2/lib/python2.7/site-packages')


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%autosave 0
from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=3)

import sys
sys.path.insert(0,'..')

from IPython.core.display import HTML
HTML('<style>.container { width:95% !important; }</style><style>.output_png {display: table-cell;text-align: center;vertical-align: middle;}</style>')


# Boxplot for reactivity values

In [ ]:
# LOAD DATA
#{seqname:[shapearray, sequence, structure]}

import rna_tools.rna_io as rio
def getdata():
    return rio.get_all_data("../data/RNA16.react","../data/RNA16.dbn")  # {key: rea, seq, stru}
import matplotlib.pyplot as plt
import simushape as sim



def get_predictions(model=None):
    # make a model 
    data = getdata()
    if model == None:
        model  = sim.make_model(data,['23sRNA','R009'])
    else:
        model  = sim.make_model(data,['23sRNA','R009'],model)
    data.pop("23sRNA")
    data.pop("R009")
    keys = data.keys()
    # predict with single structure
    def get_str(seq):
        for _, data_seq, db in data.values():
            if seq == data_seq:
                return [db]
        print "SHIT"

    # predict with multiple structures
    predictions = { k: (data[k][1],sim.predict(model,data[k][1])) for k in keys  }    
    predictions2 = { k: (v[1],sim.predict(model,v[1],seq_to_db_function=get_str)) for k,v in data.items()  }

    return predictions, predictions2 









predictions, predictions2 = get_predictions()

In [ ]:
import numpy as np
# this used optimized gradient boosting model, default random forest also looks ok..
for k in keys:    
    print k
    multi, single, real = predictions[k][1],predictions2[k][1], data[k][0] 
    real = np.array(real)
    real = real[real != np.array(None)]
    
    plt.ylim((0,3))
    print [multi,single,real]
    plt.boxplot([multi,single,real],labels=['multi','single','orig'])
    plt.show()


# so far so boring, lets do more estimators

In [ ]:


pred =[predictions, predictions2]  # first entry is default (xdg optimized)

pred+=get_predictions (sim.make_forestregressor()) # regressor is the default which is the xdg..
pred+=get_predictions (sim.RandomForestRegressor()) 



In [ ]:
import numpy as np
# this used optimized gradient boosting model, default random forest also looks ok..
for k in keys:
    real= data[k][0]
    real = np.array(real)
    #real = real[real != np.array(None)]
    plt.ylim((0,3))
    stuff = map(lambda x:x[k][1],pred)+[real] # cut out all the nones, everywherw
    stuff = map (lambda x: x[real != np.array(None)] , stuff)
    plt.boxplot(stuff,labels=["m_xdg",'s_xgd','m_fo','s_fo','m_fo_def','ss_fo_def','orig'])
    plt.show()
